# Configurations

In [ ]:
# These are configuration variable used by the notebook

# Directory that contains the expenses data
DATA_PATH = "./data"

# Person name that the expenses will be extracted from Splitwise csv
PERSON_WHO_PAYS = "Lucas Alencar"

# Category Map from Nubank/Splitwise to official categories
# This map will be used to convert a category to another one
CATEGORY_CONVERSION_TABLE = {
    'Eletricidade': 'Casa',
    'Aluguel': 'Casa',
    'Móveis': 'Casa',
    'Jantar fora': 'Restaurante',
    'Supermercado': 'Mercado',
    'Eletrônicos': 'Compras',
    'Vestuário': 'Compras',
    'Produtos de limpeza': 'Compras',
    'Filmes': 'Lazer',
    'Música': 'Lazer',
    'TV/Telefone/Internet': 'Serviços',
    'Despesas médicas': 'Saúde',
    'Táxi': 'Transporte',
    'Ônibus/trem': 'Transporte',
    'Avião': 'Viagem',
    'Hotel': 'Viagem',
    'Presentes': 'Outros',
    'Geral': 'Outros',
    'Ajuste': 'Outros'
}

from datetime import date
from date_helpers import previous_month

# Date used to make month analysis through the notebook
# BASE_DATE = date(2018, 4, 1)
BASE_DATE = previous_month(date.today())

# Year you were born
YEAR_OF_BIRTH = 1991

# The average annual return from your investments
ANNUAL_RETURN = 0.0639 # SELIC anualizada http://www.portaldefinancas.com/cdi1819.htm

# Data load and preprocess

In [ ]:
from preprocess import *

# NUBANK PREPROCESS
nubank_expenses = nubank_preprocess(read_all_csvs(DATA_PATH, 'nubank*.csv'))

# SPLITWISE PREPROCESS
splitwise = read_first_file_found(DATA_PATH, "mozi-e-eu*.csv")
splitwise = splitwise_preprocess(splitwise, PERSON_WHO_PAYS)

# OTHER EXPENSES PREPROCESS
other_expenses = read_first_file_found(DATA_PATH, "Outras*contas*Despesas*.csv")

# INCOMES PREPROCESS
incomes = read_first_file_found(DATA_PATH, "Outras*contas*Rendimentos*.csv")
incomes = incomes_preprocess(incomes)

# Data consolidation and exports
expenses = expenses_preprocess(pd.concat([nubank_expenses, splitwise, other_expenses], sort=False), CATEGORY_CONVERSION_TABLE)
export_to_sheets(expenses, DATA_PATH)
export_raw_expenses(expenses, DATA_PATH)

# Monthly Expenses

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from IPython.display import display
from monthly_expenses import *
from record_summary import total_amount_by, groupby_month

MONTHLY_BALANCE_GOAL = 0.25

# Expenses and incomes by month
exp_by_month = total_amount_by(groupby_month(expenses), expenses)

income_salary = incomes[incomes.category == 'Renda']
inc_by_month = total_amount_by(groupby_month(income_salary), income_salary)

monthly_exp = summary_expenses(exp_by_month, inc_by_month)

# Monthly averages
avg_monthly_exp = summary_expenses(exp_by_month.mean(), inc_by_month.mean())
avg_monthly_exp.index = ['Average monthly expenses']

print(" >>> Monthly balance goal:", MONTHLY_BALANCE_GOAL * 100, "%")
display(style_summary_expenses(monthly_exp, MONTHLY_BALANCE_GOAL))
display(style_summary_expenses(avg_monthly_exp, MONTHLY_BALANCE_GOAL))

# Plot expenses summary
plt.figure(figsize=(20,10))
plt.plot(monthly_exp['Expenses'] * -1, 'r', label='Expenses')
plt.plot(monthly_exp['Incomes'], 'g', label='Incomes')
plt.plot(monthly_exp['Balance'], 'b', label='Balance')
plt.legend(fontsize=20)
plt.grid()
plt.show()

# Expenses by category

In [ ]:
from date_helpers import records_for_month

from monthly_expenses import *
from plotting import bar_plot

# BASE_DATE = date(2018, 4, 1)
print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

expenses_for_month = records_for_month(expenses, BASE_DATE)
expenses_for_category_by_total = expenses_distribution(expenses_for_month)
display(style_expenses_distribution(expenses_for_category_by_total))

bar_plot(expenses_for_category_by_total)

# Investments progress

In [ ]:
# It would be interesting to have a separate sheet for investment,
# so I can add categories such as Renda Fixa or Tesouro Direto and I could know how eveything is distributed

from IPython.display import display
from monthly_investiments import *

# BASE_DATE = date(2018, 4, 1)
print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

RETURN_GOAL = 0.002

invest = incomes[incomes.category.isin(['Aplicação', 'Rendimento'])]

summary_invest = summary_investiments(invest, BASE_DATE)
display(style_summary_investments(summary_invest, RETURN_GOAL))

assets_summary = summary_assets(invest)
display(style_summary_assets(assets_summary))

# Plot line to see visualy assets progress
assets_summary.loc['2018-02':, ['Return']].plot(figsize=(20, 5), grid=True, fontsize=15, title='Return')
assets_summary.loc['2018-02':, ['Applications']].plot(figsize=(20, 5), grid=True, fontsize=15, title='Applications')
assets_summary.loc['2018-02':, ['Total']].plot(figsize=(20, 5), grid=True, fontsize=15, title='Total')

# Assets goals

In [ ]:
# PMS Patrimônio Mínimo de Sobrevivência (Reserva de emergência)
# 6 * Custo mensal

# PMR Patrimônio Mínimo Recomendado para segurança 
# Possibilita decisões mais arriscada sobre carreira ou mudanças.
# 12 * Custo mensal

# PI Patrimônio Ideal para idade e consumo 
# Dado os meus gastos mensais atuais, se eu continuar trabalhando até a aposentadoria (65 anos), 
# eu conseguiria viver apenas gastando esse dinheiro.
# 0.1 * PMR * Idade

# PNIF Patrimônio Necessário para a Independência Financeira
# Dado os meus rendimentos com investimentos, eu conseguiria manter o mesmo padrão apenas com o dinheiro investido
# PMR / Rendimento anual %

from datetime import date
from monthly_investiments import *

age = date.today().year - YEAR_OF_BIRTH

pms = 6 * exp_by_month.mean() * -1
pmr = 12 * exp_by_month.mean() * -1
pi = 0.1 * pmr * age
pnif = pmr / ANNUAL_RETURN
assets_total = invest.sum()

assets_goals = pd.DataFrame([pms, pmr, pi, pnif, assets_total], index=['PMS', 'PMR', 'PI', 'PNIF', 'Total'])
display(style_assets_goals(assets_goals, assets_total.amount))